# Goal

There's

    A) an excel document that Nathan and I created of all participants we intended to get emails. You added a sheet to the excel doc that is all the control-recipients who bounced & unsubscribed.
    B) a mailchimp export of all the PFW-recipients who bounced & unsubscribed
    C) a mailchimp export of all the eBird-recipient who bounce & unsubscribed.

The data task to perform is linking the participants with the bounce & unsubscribe statuses. That is, merging the original randomizations with 3 separate documents containing unsubscribe statuses of each randomization-arm?

In [41]:
import pandas as pd
import os
import uuid

In [42]:
!ls data

opens                    randomizations
post_experiment_analysis unsubs


In [43]:
users = pd.read_excel('data/randomizations/GBBC_2020_randomization_emails_dedupe_03102020.xlsx')
users.columns = [c.lower() for c in users.columns]

In [44]:
print(f'there are {len(users)} users')

there are 9608 users


In [45]:
#users.head()

In [46]:
users['group'].value_counts()

control        3203
feederwatch    3203
gbbc           3202
Name: group, dtype: int64

##### not sure why the randomizations had a group called gbbc. i think it should be named 'ebird' as i understand, so switching

In [47]:
users['group'] = users['group'].apply(lambda s: s.replace('gbbc', 'ebird'))

In [48]:
users['group'].value_counts()

control        3203
feederwatch    3203
ebird          3202
Name: group, dtype: int64

### merge unsubscribe and bounces

In [49]:
control = pd.read_excel('data/unsubs/GBBC_2020_randomization_emails_03102020_with_bounce_statuses.xlsx', 
                              sheet_name=3,).rename(columns={'Email':'email'})
control_bounce = control[control['Reason']=='Bounced']
control_unsub = control[control['Reason']=='Unsubscribed']
pfw_unsub = pd.read_csv('data/unsubs/mailchimp_GBBC_PFW/unsubscribed_segment_export_fc1c9a1e1e.csv',
                       usecols=['Email Address', 'UNSUB_CAMPAIGN_TITLE']).rename(columns={'Email Address':'email'})
ebird_unsub = pd.read_csv('data/unsubs/mailchimp_GBBC_eBird/unsubscribed_segment_export_8c58752a04.csv',
                          usecols=['Email Address', 'UNSUB_CAMPAIGN_TITLE']).rename(columns={'Email Address':'email'})

In [50]:
# Potentitally run this if we find out what Group A and B are:
# ebird_unsub = ebird_unsub[ebird_unsub['UNSUB_CAMPAIGN_TITLE']=='Comm_Research_eBird_2020']
# pfw_unsub = pfw_unsub[pfw_unsub['UNSUB_CAMPAIGN_TITLE']=='Comm_Research_PFW_2020']

In [51]:
print(f"There were {len(control_unsub)} unsubscribed or bounced control users")
print(f"There were {len(pfw_unsub)} unsubscribed or bounced pfw users")
print(f"There were {len(ebird_unsub)} unsubscribed or bounced ebird users")
# why are these so different? I'm guessing it because the control one is measuring a time-period of unsubscribes longer than the other unbuscribed

There were 286 unsubscribed or bounced control users
There were 4 unsubscribed or bounced pfw users
There were 12 unsubscribed or bounced ebird users


In [52]:
unsubs = pd.DataFrame(pd.concat([control_unsub["email"], pfw_unsub["email"], ebird_unsub['email']]))
unsubs['unsubscribed'] = True
bounces = pd.DataFrame(control_bounce['email'])
bounces['bounced'] = True

In [53]:
assert len(users.merge(bounces)) == len(bounces)

In [54]:
assert len(users.merge(unsubs)) == len(unsubs)

In [55]:
users = users.merge(bounces, how='left')

In [56]:
users = users.merge(unsubs, how='left')

In [57]:
print(f'there are {len(users)} users')

there are 9608 users


In [58]:
users['unsubscribed'] = users['unsubscribed'].fillna(False)
users['bounced'] = users['bounced'].fillna(False)

### Add the open states

In [59]:
OPEN_USECOLS = ['Email Address', 'Opens', '# eBird species March 2020', '# eBird checklists March 2020']
rename_columns={'Email Address':'email', 
                'Opens':'opens', 
                '# eBird species March 2020': 'num.ebird.species.march.2020', 
                '# eBird checklists March 2020': 'num.ebird.checklists.march.2020'}
control_opens = pd.read_csv('data/opens/members_Comm_Research_control_2020_opened_Apr_23_2020.csv',
                           usecols=OPEN_USECOLS).rename(columns=rename_columns)
ebird_opens = pd.read_csv('data/opens/members_Comm_Research_eBird_2020_opened_Apr_23_2020.csv',
                           usecols=OPEN_USECOLS).rename(columns=rename_columns)
pfw_opens = pd.read_csv('data/opens/members_Comm_Research_PFW_2020_opened_Apr_23_2020.csv',
                           usecols=OPEN_USECOLS).rename(columns=rename_columns)
opens = pd.concat((control_opens, ebird_opens, pfw_opens))

In [60]:
users = users.merge(opens, how='left')

In [61]:
print(f'there are {len(users)} users')

there are 9608 users


In [62]:
users['opens'] = users['opens'].fillna(0)

In [63]:
#users.tail()

### verification and summary stats

In [64]:
users['group'].value_counts()

control        3203
feederwatch    3203
ebird          3202
Name: group, dtype: int64

In [65]:
users.groupby(['group','bounced','unsubscribed']).size()

group        bounced  unsubscribed
control      False    False           2912
                      True             286
             True     False              5
ebird        False    False           3190
                      True              12
feederwatch  False    False           3199
                      True               4
dtype: int64

In [66]:
users.groupby(['group']).agg({'opens':pd.np.mean}).sort_index() #mean number of opens

,opens
group,
control,0.887293
ebird,0.973454
feederwatch,0.817047


In [67]:
users.groupby(['group']).agg({'opens':lambda series: pd.np.mean(series>0)}).sort_index() #mean nuber of users who opened

,opens
group,
control,0.411177
ebird,0.458776
feederwatch,0.401186


### Generate UUIDs 

In [71]:
users['uuid'] = [str(uuid.uuid4()) for _ in range(len(users.index))]

### write out data

In [72]:
OUT_COLS = ['uuid', 'group', 'bounced', 'unsubscribed', 'opens', 'num.ebird.species.march.2020', 'num.ebird.checklists.march.2020']

In [73]:
fname_stem = 'gbbc_experiment-results'
out_dir = 'data/post_experiment_analysis'
users[OUT_COLS].to_feather(os.path.join(out_dir, f'{fname_stem}.feather'))
users[OUT_COLS].to_csv(os.path.join(out_dir, f'{fname_stem}.csv'), index=False)